## Linear programming using cvxpy lib - animal feed ration problem

#### This script exemplifies a problem of formulating animal feed rations, by modeling an objective function (minimum price), based on decision variables (ingredients' percentage) and some restrictions regarding the ingredients. The objective is to find the percentage or mass of each ingredient (totalling 1=100% or 1kg), so that the minimum price (cost) is reached for the final ration mixture. Part of restrictions concerns to the minimum protein and calcium at the final mixture (in percentage), considering protein and calcium content by each individual ingredient (per kg). After modeling, the solver is used to solve the problem and the results are returned, with the final minimum price of the ration (per kg) (objective function's value), as well as the percentage of each ingredient at that final feed product (decision variables' values). Here, no evolutionary computation is done (including genetic algorithms), only traditional deterministic linear programming, returning always the same optimal result. For a bigger number of variables, this problem would turn difficult to be computed, if not impossible. Here, only three decision variables are used.

In [2]:
pip install cvxpy --quiet

Note: you may need to restart the kernel to use updated packages.


In [46]:
import cvxpy as cp
import numpy as np

In [47]:
# declares the decision variables, as empty placeholders, inside a Variable object (similar to what's done with solver at 
# spreadsheets). These variables' placeholders will be filled after the solver solution is calculated. All other dependent 
# variables, calculated from the decision variables, for example, the objective function's and restrictions' values, will then 
# be calculated and tested against the restrictions' conditions...
x_variables = cp.Variable(3, nonneg=True)
# => bone percentage (or mass, as a Kg fraction) => x1
# => soy percentage (or mass, as a Kg fraction)  => x2
# => fish percentage (or mass, as a Kg fraction) => x3
# x_variables is NOT an Ndarray, it's an specific object from cvxpy lib called Variable. Therefore, any operation on it, as
# the sum() of its elements, must be done through the cvxpy lib own methods (which will know how to deal with Variable objects).

# sets the objective function. First creates a cost array to store each decision variable price (per Kg). Next, defines the 
# objective function as a minimization problem, having as addends the scalar multiplication between each cost_array element 
# number by each respective x_variables decision variable number, i.e., the price per Kg by the percentage (or Kg fraction) 
# of each ingredient product, respectively.
# After the decision variables are solved, and it's found the optimal composition of ingredients (to minimize the total price), 
# the objective function can then be calculated. Here, it'll be the minimum price for the final animal feed ration per Kg, while
# the x_variables values - the decision variables - will be the ingredients' composition (as a fraction) per Kg of the final 
# ration.
cost_array = np.array([0.56, 0.81, 0.46])
objective_function = cp.Minimize(cost_array @ x_variables)

# other than the non-negative restriction already set above, we have these other restrictions set here:
# => protein per Kg of each ingredient times the percentage (or Kg fraction) of each ingredient in the ration, must be at least 
# 0.3, that is, at least 0.3 Kg (or 300g) of total protein per Kg of final ration, or 30% of total protein at the final product
# => calcium per Kg of each ingredient times the percentage (or Kg fraction) of each ingredient in the ration, must be at least 
# 0.5, that is, at least 0.5 Kg (or 500g) of total calcium per Kg of final ration, or 50% of total calcium at the final product
# => the sum of all x_variables values, i.e., the sum of the values of the decision variables (percentages or Kg fraction of each 
# ingredient) must be 1 (=100% or 1 Kg per Kg of the final ration composition)
restriction_array = [
    np.array([0.2,0.5,0.4]) @ x_variables >= 0.3,
    np.array([0.6,0.4,0.4]) @ x_variables >= 0.5,
    cp.sum(x_variables) == 1 # do not use numpy methods with cvxpy Variable objects, use cvxpy own methods
]

problem = cp.Problem(objective_function, restriction_array)
problem.solve()

np.float64(0.5100000000722827)

In [48]:
# Showing the results stored at the problem object, including the status of the problem resolution, the objective function value 
# (lowest possible cost for the ration per Kg), and the mass and percentage of each product for that final optimal ration formula
print("Results:")
print(f"Status: {problem.status}")
print(f"Minimum cost: ${problem.value:.2f}")
print(f"Bone: {x_variables.value[0]:.3f} kg ({x_variables.value[0]:.1%})")
print(f"Soy: {x_variables.value[1]:.3f} kg ({x_variables.value[1]:.1%})")
print(f"Fish: {x_variables.value[2]:.3f} kg ({x_variables.value[2]:.1%})")


Results:
Status: optimal
Minimum cost: $0.51
Bone: 0.500 kg (50.0%)
Soy: 0.000 kg (0.0%)
Fish: 0.500 kg (50.0%)


These are the same optimal values and results achieved when using the Solver at Microsoft Excel or LibreOffice Calc.

#### This script solves the exact same problem of another script - [ration-problem-lp-pulp.ipynb](./ration-problem-lp-pulp.ipynb) - at this same repository. Except for the libs used, all the rest is the same, including the results. Compare the two scripts for this same problem when using cvxpy and pulp libs. The difference, basically, is that, with cvxpy lib, we can set the decision variables, and later use a numpy ndarray to store the coefficients of the objective function and restrictions... and then simply do a scalar multiplication between these ndarrays with the Variable object containing the decision variables. While with the pulp lib, we have to use String concatenations of expressions already including the coefficients and the decision variables all the time... this might work OK for short number of variables' problems, but, for large number of variables, working with ndarrays and its operations (e.g. scalar multiplication or summation) is quite better and less error-prone.